# Run Distribution Learning Benchmark

In [1]:
import wandb

from hybrid_transformer.configs.tasks.distribution_learning import DistributionLearningConfig
from hybrid_transformer.configs.trainers.trainer import TrainerConfig

from hybrid_transformer.utils.datasets.auto import AutoDataset
from hybrid_transformer.utils.tokenizers.auto import AutoTokenizer

from hybrid_transformer.utils.runtime import set_seed, get_device

from hybrid_transformer.configs.models.model import ModelConfig
from hybrid_transformer.models.auto import AutoModel

from hybrid_transformer.configs.trainers.trainer import TrainerConfig
from hybrid_transformer.trainers.trainer import Trainer

%load_ext autoreload
%autoreload 2

/home/adam/miniconda3/envs/hybrid-transformer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-25 15:51:43.343402: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-25 15:51:43.345171: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-25 15:51:43.368987: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-25 15:51:43.369004: E external/local_xla/xla/stream_executor/cu

In [2]:
# 1. Args and Config 

args = {}
args['out_dir'] = 'guacamol'
args['experiment_config'] = 'guacamol' # GPT or HT


task_config = DistributionLearningConfig(validate=False, split='test', target_label='zale', out_dir='./results/test/')
model_config = ModelConfig(num_layers=2)
trainer_config = TrainerConfig(batch_size=8, eval_iters=2)

dataset = AutoDataset.from_config(task_config)
tokenizer = AutoTokenizer.from_config(task_config)
model = AutoModel.from_config(model_config)
trainer = Trainer(config=trainer_config, model=model, train_dataset=dataset, eval_dataset=dataset, tokenizer=tokenizer)

number of parameters: 38.06M
Running on a single device!
tokens per iteration will be: 40,960
Using cuda device
compiling the model... (takes a ~minute)
num decayed parameter tensors: 63, with 38,115,840 parameters
num non-decayed parameter tensors: 25, with 12,800 parameters
using fused AdamW: True


In [3]:
trainer.train()

BackendCompilerFailed: backend='inductor' raised:
RuntimeError: Cannot find a working triton installation. More information on installing Triton can be found at https://github.com/openai/triton

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True
